In [ ]:
%%capture

!git clone https://github.com/hudeven/text
!git clone https://github.com/facebookresearch/pytext


In [ ]:
%%capture

!pip install --upgrade --force-reinstall ./text/stl_text/dataframes/datasets;
!pip install --upgrade --force-reinstall ./text;

!pip install --upgrade --force-reinstall ./pytext;
!pip install --upgrade --force-reinstall tensorboard;

!pip install --upgrade --force-reinstall --pre torch torchtext -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html;
!pip install --upgrade --force-reinstall datasets;

!pip install pytorch-lightning

In [1]:
import os

import torch
from pytorch_lightning import Trainer
from stl_text.ops.utils.arrow import convert_csv_to_arrow
from stl_text.datamodule import DocClassificationDataModule
from task import XlmrDocClassificationTask

Install apex from https://github.com/NVIDIA/apex/.


In [2]:
# convert csv to arrow format (only required for the first time)
data_path = "./glue_sst2_tiny"
for split in ("train.tsv", "valid.tsv", "test.tsv"):
    split_path = os.path.join(data_path, split)
    convert_csv_to_arrow(split_path)

# setup datamodule
datamodule = DocClassificationDataModule(data_path=data_path, batch_size=8)
datamodule.setup("fit")

# construct task
task = XlmrDocClassificationTask(text_transform=datamodule.text_transform, num_class=2,
                                 lr=0.01)

# train model
trainer = Trainer(max_epochs=5, fast_dev_run=True)
trainer.fit(task, datamodule=datamodule)

# export to TorchScript
export_path = "/tmp/doc_task.pt1"
task.to_torchscript("/tmp/doc_task.pt1")
with open(export_path, "rb") as f:
    ts_module = torch.load(f)
    print(ts_module(text_batch=["hello world", "attend is all your need!"]))

/Users/stevenliu/opt/anaconda3/envs/fun/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Loading cached processed dataset at ./glue_sst2_tiny/train/cache-db49ecf2f701544f.arrow
Loading cached processed dataset at ./glue_sst2_tiny/train/cache-dcde595fba10e695.arrow
Loading cached processed dataset at ./glue_sst2_tiny/valid/cache-f079b8abbf5c8c2d.arrow
Loading cached processed dataset at ./glue_sst2_tiny/valid/cache-3d000a9a67cbc194.arrow
Loading cached processed dataset at ./glue_sst2_tiny/test/cache-d2d1ee7a6ae2efaf.arrow
Loading cached processed dataset at ./glue_sst2_tiny/test/cache-b09e70b8b24063db.arrow
GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU avai


tensor([[-0.9838,  0.7541],
        [-0.9837,  0.7540]], grad_fn=<AddBackward0>)
